##### STA 141B Data & Web Technologies for Data Analysis

# Lecture 14 - 02/19/26, ISS + Selenium, part II

### Announcements

### Today's topics
 - Selenium Browser
 - Finish the ISS project
 - Some Pro-Tips on hidden APIs

### Ressources
 - [AstroViewer](https://www.astroviewer.net/iss/en/observation.php)

<img src="../images/ISS_by_night_zoom.jpg" width=1200/>

<img src="../images/ISS_by_night.jpg" width=1200/>

## Reminder

Click [here](https://www.astroviewer.net/iss/en/observation.php) to access the astroviewer website!

In [ ]:
import requests
import requests_cache
import pandas as pd
import lxml.html as lx

In [ ]:
url = 'https://www.astroviewer.net/iss/en/observation.php'
headers = {
    'User - agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:142.0) Gecko/20100101 Firefox/142.0',
}

In [ ]:
response = requests.get(url)
response.raise_for_status()

In [ ]:
html = lx.fromstring(response.text)

In [ ]:
table = html.xpath('//table[contains(@class, "passDetails")]')[0]

In [ ]:
rows = table.xpath('//tbody/tr')

In [ ]:
cells = table.xpath('//tbody/tr/td')

In [ ]:
[c.text for c in cells]

In [ ]:
[c.text_content() for c in rows[0].xpath('//td')]

It conains only '...'. Have a look on the HTML!

The solution is to use selenium!

## SELENIUM

Start browser

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.quit()

Change location:

First, we have to find the input field by using its ID.

Then, we write 'Davis, CA' in the field.

Afterwards, we have to submit/press the enter key.

In [ ]:
# explicit wait
driver = webdriver.Chrome()
driver.get('https://www.astroviewer.net/iss/en/observation.php')

time.sleep(2)
input_field = driver.find_element(By.ID, "locSearch") 
input_field.send_keys("Davis, CA")

In [ ]:
input_field.send_keys(Keys.ENTER)
time.sleep(2)

In [ ]:
driver.quit()

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.astroviewer.net/iss/en/observation.php')

results_div = driver.find_element(By.ID, "passesHeader")
old_text = results_div.text
# print(old_text)

lc_box = driver.find_element(By.ID, 'locSearch')
lc_box.send_keys('Davis, CA')
lc_box.send_keys(Keys.ENTER)

# Wait for the text to change (Custom Lambda)
WebDriverWait(driver, 10).until(
    lambda d: d.find_element(By.ID, "passesHeader").text != old_text
)

print('Website has successfully loaded.')

Don't forget to close the browser, if you're done!

In [ ]:
# but we're not done yet!
# driver.quit()

Click on Buttons:

Here, we additionally want to check whether a button is selected. If not, we click on it.

In [ ]:
btn_24hrs = driver.find_element(By.ID, "24h") 

if not btn_24hrs.is_selected():
    # Click the radio button to select it
    btn_24hrs.click()
    print("Radio button selected.")
else:
    print("Radio button is already selected.")

Next, we choose our date. Note that this includes clicking on the SelectDate Button, providing a data in the format 'DD/MM/YYYY' and clicking on OK afterwards.

In [ ]:
time.sleep(5)
btn_pastPasses = driver.find_element(By.ID, "pastPassesButton")
btn_pastPasses.click()

input_field = driver.find_element(By.ID, "dateChooser") 
input_field.send_keys("10.02.2026") # This is a german website. Date must be in the format 'DD/MM/YYYY'!
input_field.send_keys(Keys.ENTER)

time.sleep(2)

driver.find_element(By.ID, "okButton").click()
time.sleep(2)

Let's find the best moment to observe the ISS. For this, we first create a list of all IDs of the shown passes:

In [ ]:
import lxml.html as lx

html = lx.fromstring(driver.page_source)
ids = html.xpath('//div[@id = "passesList"]/*[contains(@class, "passSummary")]/@id') # this returns all ids of elements of the class passSummary that followed a certain div.

In [ ]:
ids

In [ ]:
driver.find_element(By.ID, ids[4]).click() # lets click on the best observation

Next, we want to store all shown data: The date, the time, the brightness, etc.:

In [ ]:
date = driver.find_elements(By.ID, 'detailDate')[0].text

In [ ]:
cells = driver.find_elements(By.XPATH, '//table[contains(@class, "passDetails")]/tbody/tr/td')

In [ ]:
content = [c.text for c in cells]

In [ ]:
content

We don't need the last two cells.

In [ ]:
content[:-2]

Create a function that returns all the data for one observation:

In [ ]:
def get_details(nr):
    driver.find_element(By.ID, ids[nr]).click()
    date = driver.find_elements(By.ID, 'detailDate')[0].text
    cells = driver.find_elements(By.XPATH, '//table[contains(@class, "passDetails")]/tbody/tr/td')
    content = [c.text for c in cells]
    time.sleep(0.2)
    return({ids[nr]: [date] + content[:-2]})

In [ ]:
get_details(2)

Next, let us combine all information of all observations into one DataFrame.

In [ ]:
df = pd.DataFrame({key: val for i in range(len(ids)) for key,val in get_details(i).items()}).T

In [ ]:
df.columns = ['Date'] + [t + u for u in ['Time', 'Direction', 'Altitude'] for t in ['Begin', 'Max', 'End']]+['Brightness']

In [ ]:
df

Sort the df starting with the best observation.

In [ ]:
df_sorted = df.sort_values(by='Brightness', ascending=False)

In [ ]:
df_sorted

In [ ]:
print(df_sorted.iloc[0])

Combine everything from before:

In [ ]:
url = "https://www.astroviewer.net/iss/en/observation.php"

date = '11/02/2026'

def get_details(nr):
    time.sleep(1)
    driver.find_element(By.ID, ids[nr]).click()
    date = driver.find_elements(By.ID, 'detailDate')[0].text
    cells = driver.find_elements(By.XPATH, '//table[contains(@class, "passDetails")]/tbody/tr/td')
    content = [c.text for c in cells]
    return({ids[nr]: [date] + content[:-2]})

with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get(url)
    input_field = driver.find_element(By.ID, "locSearch") 
    input_field.send_keys("Davis, CA")
    input_field.send_keys(Keys.ENTER)
    time.sleep(2)

    btn_24hrs = driver.find_element(By.ID, "24h") 

    if not btn_24hrs.is_selected():
        # Click the radio button to select it
        btn_24hrs.click()
        print("Radio button selected.")
    else:
        print("Radio button is already selected.")

    time.sleep(5)
    btn_pastPasses = driver.find_element(By.ID, "pastPassesButton")
    btn_pastPasses.click()

    input_field = driver.find_element(By.ID, "dateChooser") 
    input_field.send_keys(date)
    input_field.send_keys(Keys.ENTER)

    time.sleep(2)
    
    driver.find_element(By.ID, "okButton").click()
    time.sleep(2)

    html = lx.fromstring(driver.page_source)
    ids = html.xpath('//div[@id = "passesList"]/*[contains(@class, "passSummary")]/@id')
    
    df = pd.DataFrame({key: val for i in range(len(ids)) for key,val in get_details(i).items()}).T
    df.columns = ['Date'] + [t + u for u in ['Time', 'Direction', 'Altitude'] for t in ['Begin', 'Max', 'End']]+['Brightness']
    df_sorted = df.sort_values(by='Brightness', ascending=False)
    
    print(df_sorted.iloc[0])
    time.sleep(5)

df_sorted

## Undocumented API

Alternatively, we can also use the undocumented API in the background!

In [ ]:
result = requests.get(url = 'https://www.astroviewer.net/iss/ws/predictor.php', params = {
    'sat': '25544',
    'name': 'Davis',
    'lon': '-121.7390',
    'lat': '38.5436',
    'tz': 'America/Los_Angeles'
})

result.url
result.raise_for_status()

In [ ]:
html = lx.fromstring(result.text)
result.json()